In [ ]:
# Tratamento do arquivo inf_diario e gravando dados no banco.

In [49]:
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "postgres"
}

In [31]:
import os
from zipfile import ZipFile
import requests
import re

In [24]:
url = "http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202103.csv"
file_name = "inf_diario_fi_202103.csv"
temp_dir = "tmp"


In [25]:
def download_file(url: str, filename: str, temp_dir: str) -> dict:
    """
    Download file from url and save on temp_dir
    """
    temp_file = f"{temp_dir}/{filename}"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(temp_file, "wb") as new_file:
            for partial in response.iter_content(chunk_size=256):
                new_file.write(partial)
        return {"file_path": temp_file, "file_name": filename}
    else:
        response.raise_for_status()
        
response = download_file(url, file_name, temp_dir)

print(response)


{'file_path': 'tmp/inf_diario_fi_202103.csv', 'file_name': 'inf_diario_fi_202103.csv'}


In [39]:
def format_doc(doc):
    '''
        Funcão que formata CPF e CNPJ
    '''
    if pd.isna(doc):
        return None
    doc = re.sub(r"[^0-9]", "", doc)
    return doc.zfill(14)

In [27]:
import pandas as pd

def open_file(file: str):
    return pd.read_csv(
        file,
        sep=";",
        encoding="latin-1",
    )

In [42]:
# Abrindo o arquivo com pandas
df = open_file(response.get('file_path'))

In [43]:
# Trandando os dados do DataFrame
# - 1 Filtrar por tipo de Fundo (Validar com Lucas)
# - 1 Limpar dados vazios (Validar com Lucas)
# - 2 Formatando o CPF 

df = df.loc[
        (df["TP_FUNDO"] == "FI")
        | (df["TP_FUNDO"] == "FIP")
        | (df["TP_FUNDO"] == "FIIM")
    ]

# Format documents
df["CNPJ_FUNDO"] = df["CNPJ_FUNDO"].apply(format_doc)

# Rename columns
rename_map = {
    "CNPJ_FUNDO": "fund_doc",
    "DT_COMPTC": "competency_date",
    "VL_TOTAL": "amount",
    "VL_QUOTA": "share_value",
    "VL_PATRIM_LIQ": "equity_value",
    "NR_COTST": "quotaholder_number",
}
df = df.rename(columns=rename_map)

# Columns to keep
cols_keep = [
    "fund_doc",
    "competency_date",
    "amount",
    "share_value",
    "equity_value",
    "quotaholder_number",
]
df = df[cols_keep]

,fund_doc,competency_date,amount,share_value,equity_value,quotaholder_number
0,00017024000153,2021-03-01,1090537.22,27.490241,1088117.69,1
1,00017024000153,2021-03-02,1090640.42,27.491506,1088167.76,1
2,00017024000153,2021-03-03,1090790.59,27.491962,1088185.80,1
3,00017024000153,2021-03-04,1090863.16,27.493604,1088250.80,1
4,00017024000153,2021-03-05,1090671.15,27.493286,1088238.21,1
...,...,...,...,...,...,...
460356,97929213000134,2021-03-25,70915137.12,9.330402,70904721.58,2
460357,97929213000134,2021-03-26,71174125.22,9.364430,71163311.18,2
460358,97929213000134,2021-03-29,71084799.51,9.352623,71073587.28,2
460359,97929213000134,2021-03-30,71263147.15,9.376039,71251536.05,2


In [53]:
from sqlalchemy import create_engine
connect = "postgresql+psycopg2://%s:%s@%s:54325/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
def to_alchemy(df):
    engine = create_engine(connect)
    df.to_sql(
        'cvm_ckan.fi_inf_diario', 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
    print("=== to_sql() done with sqlalchemy ===")

In [54]:
to_alchemy(df)

to_sql() done (sqlalchemy)
